In [ ]:
# !pip install folium  # uncomment to install folium package

# Instructions

1. Download your Google Maps timeline data
    - Go to [maps.google.com](https://maps.google.com/)
    - Navigate to `Your data in Maps`
    - Download `Location History`
2. Replace `file_name` below with the filepath to `records.json` file.
3. Replace `state_date`, `end_date`, `center` below.

In [ ]:
file_name = "records.json"

In [ ]:
start_date = "2023-01-01"  # YYYY-MM-DD
end_date = "2023-06-01"  # YYYY-MM-DD
center = [36.2048, 138.2529]  # lat and long map center

# Create the map

In [ ]:
import json
import folium
import pandas as pd
import matplotlib.pyplot as plt
from pytz import utc

from matplotlib.colors import Normalize, to_hex
from matplotlib.cm import ScalarMappable

In [ ]:
with open(file_name, 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data['locations'])

# Parse the timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'], format='mixed', errors='coerce')

# Convert latitudeE7 and longitudeE7 to their proper values
df['latitude'] = df['latitudeE7'] / 1e7
df['longitude'] = df['longitudeE7'] / 1e7

# Parse the timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Keep only the required columns
df = df[['latitude', 'longitude', 'timestamp']]

df = df.dropna(subset=['timestamp'])

In [ ]:
# Define start and end dates
start_datetime = pd.to_datetime(start_date).replace(tzinfo=utc)
end_datetime = pd.to_datetime(end_date).replace(tzinfo=utc)

# Normalize timestamps into a 0-1 range
df['timestamp_norm'] = (df['timestamp'] - start_datetime) / (end_datetime - start_datetime)

# Define colormap
cmap = plt.get_cmap('Spectral')

# Create a normalizer object
norm = Normalize(vmin=0, vmax=1)

# Create a scalar mappable object
sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

# Map normalized timestamps to colors
df['color'] = df['timestamp_norm'].apply(lambda x: sm.to_rgba(x))

# Filter out rows with timestamps outside the desired date range
df = df[(df['timestamp'] >= start_datetime) & (df['timestamp'] <= end_datetime)]

# Convert RGBA colors to hexadecimal
df['color'] = df['timestamp_norm'].apply(lambda x: to_hex(sm.to_rgba(x)))

In [ ]:
# Initialize a map centered around Japan
m = folium.Map(location=center, zoom_start=5, prefer_canvas=True, tiles='CartoDBPositronNoLabels')

# Add each coordinate to the map
for index, row in df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=1,
        color=row['color'],
        fill=True,
        fill_color=row['color']
    ).add_to(m)

In [ ]:
# Save the map
m.save('map.html')

In [ ]:
# Display the map
# Warning: can be slow in a notebook
m